# Tools Checking

In [24]:
from sec_api import QueryApi, XbrlApi
import os
import requests
import html2text
import re

In [ ]:
stock_name = "AAPL"
query_api = QueryApi(api_key=os.getenv("SEC_API_API_KEY"))

In [6]:
# os.getenv("SEC_API_API_KEY")

In [29]:
query = {
    "query": {
        "query_string": {
                "query": f"ticker:{stock_name} AND formType:\"10-K\"",
            }
        },
        "from": 0,
        "size": 1,
        "sort": [
            {
                "filedAt": {
                    "order": "desc"
                }
            }
        ]
}

filings = query_api.get_filings(query=query)['filings']
if len(filings) == 0:
    print("No filings found for this stock")

print("Filings", len(filings))


url = filings[0]['linkToFilingDetails']
print("URL", url)

headers = {
    "User-Agent": "sehgal.sahil0786@gmail.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "www.sec.gov"
}

response = requests.get(url, headers=headers)

print(response)

response.raise_for_status()
h = html2text.HTML2Text()
h.ignore_links = False
text = h.handle(response.content.decode('utf-8'))

text = re.sub(r"[^a-zA-Z0-9\s\n]", "", text)
print(text[:100])

Filings 1
URL https://www.sec.gov/Archives/edgar/data/320193/000032019325000079/aapl-20250927.htm
<Response [200]>
false2025FY0000320193P1YP1YP1YP1Yhttpfasborgus
gaap2025LongTermDebtNoncurrenthttpfasborgus
gaap2025L


In [ ]:
xbrlApi = XbrlApi(os.getenv("SEC_API_API_KEY"))

# URL of Google's 10-K filings
url_10k = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'

xbrl_json = xbrlApi.xbrl_to_json(htm_url=url_10k)

In [18]:
import pandas as pd 

# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 


income_statement_google = get_income_statement(xbrl_json)


print("Income statement from Google's 2022 10-K filing as dataframe")
print('------------------------------------------------------------')
income_statement_google

Income statement from Google's 2022 10-K filing as dataframe
------------------------------------------------------------


,2020-01-01-2020-12-31,2021-01-01-2021-12-31,2022-01-01-2022-12-31
RevenueFromContractWithCustomerExcludingAssessedTax,182527000000,257637000000,282836000000
CostOfRevenue,84732000000,110939000000,126203000000
ResearchAndDevelopmentExpense,27573000000,31562000000,39500000000
SellingAndMarketingExpense,17946000000,22912000000,26567000000
GeneralAndAdministrativeExpense,11052000000,13510000000,15724000000
CostsAndExpenses,141303000000,178923000000,207994000000
OperatingIncomeLoss,41224000000,78714000000,74842000000
NonoperatingIncomeExpense,6858000000,12020000000,-3514000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,48082000000,90734000000,71328000000
IncomeTaxExpenseBenefit,7813000000,14701000000,11356000000


# Testing Calculator Tool

In [34]:
from crewai.tools import BaseTool
import ast
import operator
import re

In [35]:
allowed_operators = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Pow: operator.pow,
    ast.Mod: operator.mod,
    ast.UAdd: operator.pos,
    ast.USub: operator.neg,
}

In [39]:
expression = "1+2*3"
# Parse the expression and validate it
if not re.match(r'^[0-9+\-*/().% ]+$', expression):
    raise ValueError(f"Invalid characters in mathematical expression: {expression}")

In [40]:
tree = ast.parse(expression, mode="eval")

In [41]:
def _eval_node(node):
    if isinstance(node, ast.Expression):
        return _eval_node(node.body)
    elif isinstance(node, ast.Constant):
        return node.value
    elif isinstance(node, ast.BinOp):
        left = _eval_node(node.left)
        right = _eval_node(node.right)
        op = allowed_operators.get(type(node.op))
        if op is None:
            raise ValueError(f"Unsupported operator: {type(node.op).__name__}")
        return op(left, right)
    elif isinstance(node, ast.UnaryOp):
        operand = _eval_node(node.operand)
        op = allowed_operators.get(type(node.op))
        if op is None:
            raise ValueError(f"Unsupported operator: {type(node.op).__name__}")
        return op(operand)
    else:
        raise ValueError(f"Unsupported node type: {type(node).__name__}")
    result = _eval_node(tree)
    return result

In [43]:
_eval_node(tree)

7